In [1]:
from sklearn import svm
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from glob import glob
import cv2
import os
import numpy as np

N_CLUSTERS = 5


def load_image(path):
    #read image
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img.reshape((img.shape[0] * img.shape[1], 3))
    return img


def fex(img, n_clusters):
    #using k-means to cluster pixels
    kmeans = KMeans(n_clusters)
    kmeans.fit(img)

    #the cluster centers are our dominant colour_clusters.
    colour_clusters = kmeans.cluster_centers_
    colour_clusters = colour_clusters.astype(int)

    n_labels = np.arange(0, n_clusters+1)
    (hist, _) = np.histogram(kmeans.labels_, bins=n_labels)
    hist = hist.astype("float")
    hist /= hist.sum()

    #descending order sorting as per frequency count
    colour_clusters = colour_clusters[(-hist).argsort()]
    hist = hist[(-hist).argsort()] 

    #creating empty chart
    chart = np.zeros((50, 500, 3), np.uint8)
    start = 0

    cluster_frequencies = np.array([[]])

    #creating color rectangles
    for i in range(n_clusters):
        end = start + hist[i] * 500

        cluster_frequencies = np.append(cluster_frequencies, hist[i])

        #getting rgb values
        r = colour_clusters[i][0]
        g = colour_clusters[i][1]
        b = colour_clusters[i][2]

        #using cv2.rectangle to plot colour_clusters
        cv2.rectangle(chart, (int(start), 0), (int(end), 50), (r,g,b), -1)
        start = end
    #     print
    #     print "R ", r
    #     print "G " , g
    #     print "B ", b
    #     print

    #display chart
#     plt.figure()
#     plt.axis("off")
#     plt.imshow(chart)
#     plt.show()
#     print 
#     print "acluster_frequencies dimension ", cluster_frequencies.ndim
#     print cluster_frequencies
    outarr = np.zeros(len(colour_clusters)*4)
    for i in range(len(colour_clusters)):
        outarr[4*i:4*i+3] = colour_clusters[i]
        outarr[4*(i+1)-1] = cluster_frequencies[i]

    return outarr

main_path = '/Users/frodo/Desktop/cock/'

features = []
labels = []

for directory in glob(main_path+'*'):
    print directory
    for img in os.listdir(directory):
        img_path = os.path.join(directory,img)
        img = load_image(img_path)
        feature = fex(img,N_CLUSTERS)
        features.append(feature)
        label = directory.split('/')[-1]
        labels.append(label)
        print label
        
features = np.array(features)
labels = np.array(labels)

/Users/frodo/Desktop/cock/cazzo
cazzo
cazzo
cazzo
/Users/frodo/Desktop/cock/merda
merda
merda
merda
/Users/frodo/Desktop/cock/cielo
cielo
cielo
cielo


In [3]:
clf = svm.SVC()
clf.fit(features, labels)

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [22]:
img_test = load_image('/Users/frodo/Desktop/1529005200_0_2018-06-14-20-40-00.jpg')
test_feature = np.zeros(len(features)*4)
test_features = []
test_feature = fex(img_test,N_CLUSTERS)
test_features.append(test_feature)
test_features = np.array(test_features)

# print test_features
predicted_labels = []
predicted_label = clf.predict(test_features)
print predicted_label
predicted_labels.append(predicted_label)
predicted_labels = np.array(predicted_labels)

['cielo']
